# Análisis de Evasión de Clientes (Churn) - TelecomX

Este notebook explora y analiza los factores que influyen en la evasión de clientes (churn) en una empresa de telecomunicaciones. Se abordan todas las etapas del proceso de ciencia de datos: extracción, limpieza, transformación, análisis exploratorio y visualización, con el objetivo de identificar patrones y proponer estrategias de retención.

---

In [ ]:
# Importación de librerías principales
# ----------------------------------
# pandas: manipulación y análisis de datos
# matplotlib y seaborn: visualización de datos
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


: 

## 1. Extracción de Datos

En esta sección se importan los datos originales y se realiza una primera inspección para conocer la estructura y el contenido del dataset.

In [ ]:
# Cargar los datos desde el archivo JSON
# Asegúrate de que la ruta sea correcta según tu entorno
# El archivo debe estar en la carpeta notebooks/
df = pd.read_json('TelecomX_Data.json')

# Vista rápida de los primeros registros para validar la carga
df.head()

In [ ]:
df = pd.read_json('/content/TelecomX_Data.json')
df.head()

## 2. Transformación de Datos

Se realiza limpieza, normalización y transformación de las variables para preparar el dataset para el análisis. Incluye manejo de valores nulos, duplicados, creación de nuevas variables y estandarización de nombres.

### Conoce el cojunto de datos

In [ ]:
# Inspección general de la estructura del DataFrame
# Muestra información sobre columnas, tipos de datos y valores nulos
df.info()

In [ ]:
# Visualizar los tipos de datos de cada columna
# Útil para identificar variables categóricas y numéricas
df.dtypes

### Valores nulos

In [ ]:
# Detección de valores nulos en cada columna
# Importante para decidir estrategias de limpieza de datos
df.isnull().sum()

In [ ]:
# Verificar si existen IDs de cliente duplicados
# Un ID duplicado puede indicar un problema de integridad en los datos
df['customerID'].duplicated().sum()

### Normalizamos todas las columnas

In [ ]:
# Normalización de sub-diccionarios y unión al DataFrame principal
# Cada sección (customer, phone, internet, account) se expande a columnas propias

df_customer = pd.json_normalize(df['customer'])  # Datos personales

df_phone = pd.json_normalize(df['phone'])        # Servicios telefónicos

df_internet = pd.json_normalize(df['internet'])  # Servicios de internet

df_account = pd.json_normalize(df['account'])    # Información de cuenta

# Unir todas las columnas relevantes en un solo DataFrame
# Se conservan las columnas clave y se eliminan duplicados

df = pd.concat([df[['customerID', 'Churn']], df_customer, df_phone, df_internet, df_account], axis=1)

df.duplicated().sum()  # Verificar duplicados en el DataFrame final

### Manejo de inconsistencias

In [ ]:
# Limpieza de la columna 'Churn'
# Se eliminan espacios y se estandariza el formato (primera letra mayúscula)
df['Churn'] = df['Churn'].str.strip().str.capitalize()

In [ ]:
# Visualizar los primeros registros tras la limpieza y transformación
df.head()

In [ ]:
df_customer.describe()

In [ ]:
df[df['tenure'] == 0].groupby('Churn').size()

In [ ]:
median_tenure = df[df['tenure'] > 0]['tenure'].median()
df.loc[(df['tenure'] == 0) & (df['Churn'] == 'No'), 'tenure'] = median_tenure


### Columna de cuentas diarias

In [ ]:
df.columns

In [ ]:

df['Cuentas_Diarias'] = df['Charges.Monthly'].fillna(0) / 30


In [ ]:
df[['Charges.Monthly', 'Cuentas_Diarias']].head()


In [ ]:
# Crear copia para seguridad (opcional)
df = df.copy()

# Convertir columnas de Sí/No a binario
cols_binarias = ['Churn', 'Partner', 'Dependents', 'PaperlessBilling']

for col in cols_binarias:
    df[col + '_Binario'] = df[col].map({'Yes': 1, 'No': 0})

# Renombrar columnas para mayor claridad
df.rename(columns={
    'Charges.Monthly': 'Facturacion_Mensual',
    'Charges.Total': 'Facturacion_Total',
    'tenure': 'Meses_Cliente'
}, inplace=True)

# Revisar cómo quedó
print(df.head())


## 3. Análisis y Carga de Datos

En esta etapa se exploran los datos transformados, se generan estadísticas descriptivas y se visualizan patrones relevantes para el churn.

### Analisis descritivo

In [ ]:
# Estadísticas descriptivas para todas las columnas numéricas
print(df.describe())


### Distribucion de evasion

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo
sns.set(style="whitegrid", font_scale=1.1)
custom_palette = ['#4F8EF7', '#F76E6E']  # Azul y rojo suave

# Contar valores únicos de Churn
churn_counts = df['Churn'].value_counts()

# Mostrar tabla de proporciones
print("Conteo de clientes por Churn:\n", churn_counts)
print("\nProporción:\n", churn_counts / len(df))

# 📊 1️⃣ Gráfico de barras personalizado
plt.figure(figsize=(7,5))
ax = sns.countplot(data=df, x='Churn', palette=custom_palette, edgecolor='black', linewidth=2)
plt.title('Distribución de Evasión de Clientes (Churn)', fontsize=15, fontweight='bold')
plt.xlabel('Churn', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=11, color='black', xytext=(0, 10), textcoords='offset points')
plt.tight_layout()
plt.show()

# 🥧 2️⃣ Gráfico de pastel personalizado
plt.figure(figsize=(7,7))
wedges, texts, autotexts = plt.pie(
    churn_counts,
    labels=churn_counts.index,
    autopct='%1.1f%%',
    colors=custom_palette,
    startangle=120,
    textprops={'fontsize': 13},
    wedgeprops={'edgecolor': 'white', 'linewidth': 2}
)
plt.setp(autotexts, weight='bold', color='white')
plt.title('Proporción de Clientes que se Van vs. se Quedan', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()


### Recuento de evasión por variables categóricas

#### Distribución de Churn por Género

In [ ]:
# Gráfico de barras: Evasión de Clientes por Género
plt.figure(figsize=(8,5))
ax = sns.countplot(data=df, x='gender', hue='Churn', palette=custom_palette, edgecolor='black', linewidth=2)
plt.title('Evasión de Clientes por Género', fontsize=14, fontweight='bold')
plt.xlabel('Género', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.legend(title='Churn', fontsize=11, title_fontsize=12)
for p in ax.patches:
    height = int(p.get_height())
    if height > 0:
        ax.annotate(f'{height}', (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 8), textcoords='offset points')
plt.tight_layout()
plt.show()

# Tabla de proporción por género y churn
print(df.groupby(['gender', 'Churn']).size().unstack())


#### Distribución de Churn por Tipo de Contrato

In [ ]:
# Gráfico de barras: Evasión de Clientes por Tipo de Contrato
plt.figure(figsize=(10,5))
ax = sns.countplot(data=df, x='Contract', hue='Churn', palette=custom_palette, edgecolor='black', linewidth=2)
plt.title('Evasión de Clientes por Tipo de Contrato', fontsize=14, fontweight='bold')
plt.xlabel('Tipo de Contrato', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xticks(rotation=15)
plt.legend(title='Churn', fontsize=11, title_fontsize=12)
for p in ax.patches:
    height = int(p.get_height())
    if height > 0:
        ax.annotate(f'{height}', (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 8), textcoords='offset points')
plt.tight_layout()
plt.show()

# Tabla de proporción por tipo de contrato y churn
print(df.groupby(['Contract', 'Churn']).size().unstack())


#### Distribución de Churn por Método de Pago

In [ ]:
# Gráfico de barras: Evasión de Clientes por Método de Pago
plt.figure(figsize=(12,5))
ax = sns.countplot(data=df, x='PaymentMethod', hue='Churn', palette=custom_palette, edgecolor='black', linewidth=2)
plt.title('Evasión de Clientes por Método de Pago', fontsize=14, fontweight='bold')
plt.xlabel('Método de Pago', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xticks(rotation=30)
plt.legend(title='Churn', fontsize=11, title_fontsize=12)
for p in ax.patches:
    height = int(p.get_height())
    if height > 0:
        ax.annotate(f'{height}', (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 8), textcoords='offset points')
plt.tight_layout()
plt.show()

# Tabla de proporción por método de pago y churn
print(df.groupby(['PaymentMethod', 'Churn']).size().unstack())


#### Conteo de evasión por variables numéricas

In [ ]:
import seaborn as sns

##### Boxplot: Total Gastado vs Churn

In [ ]:
print(df.columns)


In [ ]:
plt.figure(figsize=(8,5))
ax = sns.boxplot(data=df, x='Churn', y='Facturacion_Total', hue='Churn', palette=custom_palette, linewidth=2, fliersize=3, legend=False)
plt.title('Distribución de Total Gastado según Evasión', fontsize=14, fontweight='bold')
plt.xlabel('Churn', fontsize=12)
plt.ylabel('Facturación Total', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend([],[], frameon=False)  # Oculta la leyenda duplicada
plt.tight_layout()
plt.show()


##### Boxplot: Meses como cliente vs. Evasión

In [ ]:
plt.figure(figsize=(8,5))
ax = sns.boxplot(data=df, x='Churn', y='Meses_Cliente', palette=custom_palette, linewidth=2, fliersize=3)
plt.title('Distribución de Meses como Cliente según Evasión', fontsize=14, fontweight='bold')
plt.xlabel('Churn', fontsize=12)
plt.ylabel('Meses Cliente', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


##### Histograma: Facturación Total segmentada por Evasión

In [ ]:
plt.figure(figsize=(8,5))
ax = sns.histplot(data=df, x='Facturacion_Total', hue='Churn', bins=30, kde=True, palette=custom_palette, alpha=0.7, edgecolor='black')
plt.title('Histograma de Facturación Total según Evasión', fontsize=14, fontweight='bold')
plt.xlabel('Facturación Total', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.legend(title='Churn', fontsize=11, title_fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## INFORME FINAL

## 4. Informe Final y Conclusiones

Resumen de hallazgos, insights clave y recomendaciones estratégicas basadas en el análisis realizado. Esta sección sintetiza los resultados y sugiere acciones para reducir la evasión de clientes.

# Informe Final: Análisis de Evasión de Clientes (Churn)

---

## 1. Introducción
En este proyecto se analizó un conjunto de datos de clientes de una empresa de telecomunicaciones con el objetivo principal de identificar patrones asociados a la evasión de clientes (**churn**). Entender por qué los clientes cancelan el servicio permite diseñar estrategias para reducir la deserción y fidelizar usuarios.

---

## 2. Limpieza y Tratamiento de Datos
**Principales pasos realizados:**
- Importación de datos y revisión de estructura.
- Eliminación de registros duplicados y valores nulos, asegurando consistencia.
- Conversión de variables categóricas a variables binarias para facilitar el análisis (Sí / No → 1 / 0).
- Creación de nuevas columnas derivadas como:
  - **Cuentas_Diarias:** Facturación mensual dividida por 30 para estimar gasto diario.
  - **Meses_Cliente:** Ajustando el nombre de tenure.
- Estandarización de nombres de columnas para mejor legibilidad.

---

## 3. Análisis Exploratorio de Datos
**Visualizaciones y hallazgos:**
- **Distribución general de churn:**
  - Proporción de clientes que permanecen vs. clientes que abandonan.
- **Churn vs. variables categóricas:**
  - Comparativas de churn por género, tipo de contrato, método de pago, etc.
  - Revelan diferencias clave entre perfiles de clientes.
- **Churn vs. variables numéricas:**
  - Análisis de Facturacion_Total, Facturacion_Mensual y Meses_Cliente.
  - Boxplots, violinplots e histogramas evidencian cómo se comportan los montos gastados y el tiempo de permanencia respecto a la evasión.

---

## 4. Conclusiones e Insights

- **Clientes con menor tiempo de permanencia tienen mayor tendencia a cancelar.**
- **Clientes con facturación mensual más baja muestran una tasa de churn ligeramente mayor.**
- **Contratos de tipo mes a mes están más asociados con churn que contratos a largo plazo.**
- **Métodos de pago automáticos tienden a retener mejor a los clientes que pagos manuales.**

Estos patrones permiten perfilar grupos de riesgo y focalizar estrategias.

---

## 5. Recomendaciones

- Incentivar contratos de mayor plazo mediante descuentos o beneficios exclusivos.
- Diseñar programas de fidelización para clientes con baja facturación, fomentando la venta cruzada de servicios.
- Promover métodos de pago automáticos mediante campañas de concienciación o bonificaciones.
- Implementar un seguimiento proactivo a clientes con pocos meses de servicio para aumentar la retención desde el inicio.

---

##  Cierre
Este análisis es un primer paso hacia la reducción de la evasión de clientes. Implementar estrategias basadas en estos hallazgos puede traducirse en mayor lealtad, ingresos sostenibles y crecimiento empresarial.

## Extra

### Calcular matriz de correlación

In [ ]:
# Solo columnas numéricas
corr_matrix = df.select_dtypes(include=['int64', 'float64']).corr()

# Visualizar la matriz
print(corr_matrix)


### Visualizar la matriz con un mapa de calor

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('📊 Matriz de Correlación')
plt.show()


### Visualizar relaciones específicas

In [ ]:
# Revisar correlación específica
print(corr_matrix['Churn_Binario'].sort_values(ascending=False))


# 📊 Análisis de Evasión de Clientes (Churn) - Telco

Este proyecto tiene como objetivo analizar y comprender los factores que influyen en la **evasión de clientes** (*churn*) en una empresa de telecomunicaciones. Se realiza un análisis exploratorio completo, limpieza de datos, visualizaciones y correlaciones para identificar patrones clave.

---

## 🚀 Contenido

- [Descripción](#descripción)
- [Estructura del Proyecto](#estructura-del-proyecto)
- [Requisitos](#requisitos)
- [Instalación](#instalación)
- [Uso](#uso)
- [Resultados](#resultados)
- [Autor](#autor)
- [Contacto](#contacto)
- [Licencia](#licencia)

---

## 📌 Descripción

El problema de **evasión de clientes** es uno de los principales retos de las empresas de servicios. Este análisis busca identificar características comunes entre clientes que se dan de baja y generar *insights* para tomar decisiones estratégicas y reducir la tasa de churn.

El proyecto incluye:
- Limpieza y tratamiento de datos.
- Estandarización y transformación de variables.
- Creación de nuevas variables (e.g., cuentas diarias, binarización).
- Análisis exploratorio con estadísticas descriptivas.
- Visualización de patrones de evasión por variables categóricas y numéricas.
- Análisis de correlación entre variables clave.
- Conclusiones y recomendaciones estratégicas.

---

## 📂 Estructura del Proyecto

📁 churn-analysis/
│
├── README.md
├── churn_analysis.ipynb
├── 📁 data/
│ └── telco_churn.csv
└── 📁 images/
└── (gráficos generados)

yaml
Copiar
Editar

---

## ⚙️ Requisitos

- Python >= 3.8
- Pandas
- Matplotlib
- Seaborn
- Jupyter Notebook o Google Colab

---

## ✅ Instalación

1️⃣ Clona este repositorio:  
```bash
git clone https://github.com/tuusuario/churn-analysis.git
cd churn-analysis
2️⃣ Instala las dependencias necesarias:

bash
Copiar
Editar
pip install pandas matplotlib seaborn
3️⃣ Abre el notebook:

bash
Copiar
Editar
jupyter notebook churn_analysis.ipynb
▶️ Uso
Ejecuta el notebook paso a paso.

Sigue cada sección: limpieza, análisis exploratorio, visualizaciones y conclusiones.

Personaliza el análisis cambiando parámetros o columnas según tus necesidades.

📈 Resultados
Los principales resultados incluyen:

Distribución general de clientes que permanecen y clientes que se dan de baja.

Relación entre churn y variables categóricas (género, tipo de contrato, método de pago).

Relación entre churn y variables numéricas como facturación mensual, facturación total y duración del contrato.

Identificación de correlaciones clave que pueden apoyar modelos predictivos.

Conclusiones claras y recomendaciones accionables para reducir la evasión.

✏️ Autor
Jorge Henriquez Novoa
Ingeniero Electrónico con énfasis en análisis de datos
📍 Barranquilla, Colombia

📬 Contacto
Email: tu_email@example.com

LinkedIn: Tu perfil de LinkedIn